In [24]:

!pip install bitmex


In [27]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook 
bitmex_api_key = 'rvxDZwYmLtPPRhHsawgd-fWR'   
bitmex_api_secret = '-oufTuHHFLgJScsG9K3M3dSloY2opCXPfQomGcBri7BvRbsh' 
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)

def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new


def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('Done')
    return data_df

/usr/local/lib/python3.6/dist-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [30]:
get_all_bitmex("XBTUSD" , "5m" , save=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



All caught up..!


,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2015-09-25 12:05:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:10:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:15:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:20:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
2015-09-25 12:25:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-17 05:55:00+00:00,XBTUSD,11824.0,11824.5,11823.0,11823.0,460,1856069,11824.5241,1.0,15698188373,156.981884,1856069.0
2020-08-17 06:00:00+00:00,XBTUSD,11823.0,11823.5,11809.5,11813.0,1243,5022253,11814.7448,50.0,42511310173,425.113102,5022253.0
2020-08-17 06:05:00+00:00,XBTUSD,11813.0,11818.0,11803.0,11817.5,1522,6111034,11807.7695,10.0,51756955577,517.569556,6111034.0


In [33]:
get_all_bitmex("XRPM19" , "1m" , save=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



All caught up..!


,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2019-03-15 08:32:00+00:00,XRPM19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.000000
2019-03-15 08:33:00+00:00,XRPM19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.000000
2019-03-15 08:34:00+00:00,XRPM19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.000000
2019-03-15 08:35:00+00:00,XRPM19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.000000
2019-03-15 08:36:00+00:00,XRPM19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-28 11:57:00+00:00,XRPM19,0.000034,0.000034,0.000034,0.000034,19,187220,0.000034,100.0,641977980,187220.0,6.419780
2019-06-28 11:58:00+00:00,XRPM19,0.000034,0.000034,0.000034,0.000034,8,30501,0.000034,200.0,104588330,30501.0,1.045883
2019-06-28 11:59:00+00:00,XRPM19,0.000034,0.000034,0.000034,0.000034,9,13917,0.000034,2000.0,47721793,13917.0,0.477218
